In [8]:
import pandas as pd

In [4]:
import json
import time
from time import sleep

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

# --크롬창을 숨기고 실행-- driver에 options를 추가해주면된다
# options = webdriver.ChromeOptions()
# options.add_argument('headless')

url = 'https://map.naver.com/v5/search'
driver = webdriver.Chrome()  # 드라이버 경로
# driver = webdriver.Chrome('./chromedriver',chrome_options=options) # 크롬창 숨기기
driver.get(url)
key_word = '연희동 카페'  # 검색어

# css 찾을때 까지 10초대기
def time_wait(num, code):
    try:
        wait = WebDriverWait(driver, num).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, code)))
    except:
        print(code, '태그를 찾지 못하였습니다.')
        driver.quit()
    return wait

# frame 변경 메소드
def switch_frame(frame):
    driver.switch_to.default_content()  # frame 초기화
    driver.switch_to.frame(frame)  # frame 변경

# 페이지 다운
def page_down(num):
    body = driver.find_element(By.CSS_SELECTOR, 'body')
    body.click()
    for i in range(num):
        body.send_keys(Keys.PAGE_DOWN)

# css를 찾을때 까지 10초 대기
time_wait(10, 'div.input_box > input.input_search')

# (1) 검색창 찾기
search = driver.find_element(By.CSS_SELECTOR, 'div.input_box > input.input_search')
search.send_keys(key_word)  # 검색어 입력
search.send_keys(Keys.ENTER)  # 엔터버튼 누르기

sleep(1)

# (2) frame 변경
switch_frame('searchIframe')
page_down(40)
sleep(3)

# 카페 리스트
cafe_list = driver.find_elements(By.CSS_SELECTOR, 'li.UEzoS')
# 페이지 리스트
next_btn = driver.find_elements(By.CSS_SELECTOR, '.zRM9F> a')

# dictionary 생성
cafe_dict = {'카페정보': []}
# 시작시간
start = time.time()
print('[크롤링 시작...]')

# 크롤링 (페이지 리스트 만큼)
for btn in range(len(next_btn))[1:]:  # next_btn[0] = 이전 페이지 버튼 무시 -> [1]부터 시작
    cafe_list = driver.find_elements(By.CSS_SELECTOR, 'li.UEzoS')
    names = driver.find_elements(By.CSS_SELECTOR, '.TYaxT')  # (3) 장소명
    types = driver.find_elements(By.CSS_SELECTOR, '.KCMnt')  # (4) 장소 유형

    for data in range(len(cafe_list)):  # 카페 리스트 만큼
        print(data)

        sleep(1)
        try:
            # 지번, 도로명 초기화
            jibun_address = ''
            road_address = ''

            # (3) 카페명 가져오기
            cafe_name = names[data].text
            print(cafe_name)

            # (4) 유형
            cafe_type = types[data].text
            print(cafe_type)

            # dict에 데이터 집어넣기
            dict_temp = {
                'name': cafe_name,
                'cafe_type': cafe_type,
            }

            cafe_dict['카페정보'].append(dict_temp)
            sleep(1)

        except Exception as e:
            print(e)
            print('ERROR!' * 3)

            dict_temp = {
                'name': cafe_name,
                'cafe_type': cafe_type

            }

            cafe_dict['카페정보'].append(dict_temp)
            sleep(1)

    # 다음 페이지 버튼 누를 수 없으면 종료
    if not next_btn[-1].is_enabled():
        break

    if names[-1]:  # 마지막 카페일 경우 다음버튼 클릭
        next_btn[-1].click()

        sleep(2)

    else:
        print('페이지 인식 못함')
        break

print('[데이터 수집 완료]\n소요 시간 :', time.time() - start)
driver.quit()  # 작업이 끝나면 창을 닫는다.

# 배열에 저장
data = [cafe_dict]


KeyboardInterrupt: 

In [27]:
data_list= data[0]['카페정보']

In [119]:
import pandas as pd

df = pd.DataFrame(data_list)
df

,name,cafe_type
0,Poke all day 포케&샐러드 연희점,"다이어트,샐러드"
1,그레인서울,브런치
2,스웨이커피스테이션,"카페,디저트"
3,프로토콜 연희점,"카페,디저트"
4,보연희,"카페,디저트"
...,...,...
235,카페드댕,"카페,디저트"
236,리히트,케이크전문
237,하우스오브바이닐 연남점,"카페,디저트"
238,커피하우스,"카페,디저트"


In [120]:
unique_name= df['name'].value_counts()

In [121]:
duplicate_names= unique_name[unique_name > 1].index

In [122]:
duplicate_names

Index(['그레인서울', '쿳사 연희', '팔레트샐러드 연희점'], dtype='object', name='name')

In [123]:
del_name=[]
for name in duplicate_names: 
    del_name.append(df[df['name'] == name].index[0])

In [124]:
del_name

[1, 23, 22]

In [125]:
df.drop(del_name, inplace=True)

In [126]:
df['name'].value_counts()

name
Poke all day 포케&샐러드 연희점    1
카페도비                       1
연희롱블랙                      1
이디야커피 서대문구청점               1
가비노                        1
                          ..
베리굳베이글 연희점                 1
벨르아티                       1
에스페란자 로스터즈 연희점             1
그레인서울                      1
오버딥                        1
Name: count, Length: 237, dtype: int64

In [127]:
df.to_csv('cafelist_연희동.csv',encoding='utf-8-sig')

In [2]:
import json
import time
from time import sleep

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

# --크롬창을 숨기고 실행-- driver에 options를 추가해주면된다
# options = webdriver.ChromeOptions()
# options.add_argument('headless')

url = 'https://map.naver.com/v5/search'
driver = webdriver.Chrome()  # 드라이버 경로
# driver = webdriver.Chrome('./chromedriver',chrome_options=options) # 크롬창 숨기기
driver.get(url)
key_word = '망원동 카페'  # 검색어

# css 찾을때 까지 10초대기
def time_wait(num, code):
    try:
        wait = WebDriverWait(driver, num).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, code)))
    except:
        print(code, '태그를 찾지 못하였습니다.')
        driver.quit()
    return wait

# frame 변경 메소드
def switch_frame(frame):
    driver.switch_to.default_content()  # frame 초기화
    driver.switch_to.frame(frame)  # frame 변경

# 페이지 다운
def page_down(num):
    body = driver.find_element(By.CSS_SELECTOR, 'body')
    body.click()
    for i in range(num):
        body.send_keys(Keys.PAGE_DOWN)

# css를 찾을때 까지 10초 대기
time_wait(10, 'div.input_box > input.input_search')

# (1) 검색창 찾기
search = driver.find_element(By.CSS_SELECTOR, 'div.input_box > input.input_search')
search.send_keys(key_word)  # 검색어 입력
search.send_keys(Keys.ENTER)  # 엔터버튼 누르기

sleep(1)

# (2) frame 변경
switch_frame('searchIframe')
page_down(40)
sleep(3)

# 카페 리스트
cafe_list = driver.find_elements(By.CSS_SELECTOR, 'li.UEzoS')
# 페이지 리스트
next_btn = driver.find_elements(By.CSS_SELECTOR, '.zRM9F> a')

# dictionary 생성
cafe_dict = {'카페정보': []}
# 시작시간
start = time.time()
print('[크롤링 시작...]')

# 크롤링 (페이지 리스트 만큼)
for btn in range(len(next_btn))[1:]:  # next_btn[0] = 이전 페이지 버튼 무시 -> [1]부터 시작
    cafe_list = driver.find_elements(By.CSS_SELECTOR, 'li.UEzoS')
    names = driver.find_elements(By.CSS_SELECTOR, '.TYaxT')  # (3) 장소명
    types = driver.find_elements(By.CSS_SELECTOR, '.KCMnt')  # (4) 장소 유형

    for data in range(len(cafe_list)):  # 카페 리스트 만큼
        print(data)

        sleep(1)
        try:
            # 지번, 도로명 초기화
            jibun_address = ''
            road_address = ''

            # (3) 카페명 가져오기
            cafe_name = names[data].text
            print(cafe_name)

            # (4) 유형
            cafe_type = types[data].text
            print(cafe_type)

            # dict에 데이터 집어넣기
            dict_temp = {
                'name': cafe_name,
                'cafe_type': cafe_type,
                # 'road_address': road_address,
                # 'jibun_address': jibun_address
            }

            cafe_dict['카페정보'].append(dict_temp)
            sleep(1)

        except Exception as e:
            print(e)
            print('ERROR!' * 3)

            dict_temp = {
                'name': cafe_name,
                'cafe_type': cafe_type

            }

            cafe_dict['카페정보'].append(dict_temp)
            sleep(1)

    # 다음 페이지 버튼 누를 수 없으면 종료
    if not next_btn[-1].is_enabled():
        break

    if names[-1]:  # 마지막 카페일 경우 다음버튼 클릭
        next_btn[-1].click()

        sleep(2)

    else:
        print('페이지 인식 못함')
        break

print('[데이터 수집 완료]\n소요 시간 :', time.time() - start)
driver.quit()  # 작업이 끝나면 창을 닫는다.

# 배열에 저장
data = [cafe_dict]


[크롤링 시작...]
0
베이코닉 브런치바
브런치
1
라비뛰드
베이커리
2
어글리베이커리
베이커리
3
더바이글
베이커리
4
소설원 망원
카페,디저트
5
투떰즈업
베이커리
6
비캔드
브런치카페
7
슈크렌느
카페,디저트
8
베이크 샵 하벳
카페,디저트
9
라뚜셩트
카페,디저트
10
비전스트롤 망원점
카페,디저트
11
구황작물
카페,디저트
12
브릭베이글
베이글
13
해브어
카페,디저트
14
아브르드팡
카페,디저트
15
올웨이즈어거스트로스터스
카페
16
딜리커피 망원
카페,디저트
17
후와후와
베이커리
18
밀로밀
베이커리
19
티노마드
카페,디저트
20
베이코닉 브런치바
브런치
21
커피가게동경
카페
22
키토빵앗간
베이커리
23
마떼아
카페,디저트
24
크림데이 글루텐프리 케이크샵
케이크전문
25
포트레이트커피바
카페,디저트
26
레스피레
베이커리
27
망원 지튼
카페,디저트
28
커넥츠커피 망원
카페,디저트
29
당도
아이스크림
30
모을
카페
31
이이알티 망원
카페,디저트
32
라바즈
베이커리
33
콘웰
카페,디저트
34
두두리두팡
베이커리
35
딥블루레이크
카페,디저트
36
보통공원
카페,디저트
37
광합성카페
카페,디저트
38
모아새
카페,디저트
39
이스트우드 망원점
카페,디저트
40
카펫
카페,디저트
41
락떼스피릿
카페
42
필리커피
카페,디저트
43
구스커피앤바
카페,디저트
44
미완성식탁
카페,디저트
45
프레쎄
카페,디저트
46
블랑제리코팡
베이커리
47
카페뎀셀브즈 망원동 소아과점
카페
48
흐쎄트
베이커리
49
루아르 망원
카페,디저트
50
도쿄빙수 망원본점
빙수
51
HHSS HOUSE
카페,디저트
52
단고당
카페,디저트
53
바이어리셔
베이커리
0
스타벅스 망원한강공원점
카페
1
키오스크
카페,디저트
2
푸쉬오프 커피앤바
카페,디저트
3
엣모스피어
카페,디저트
4
히카
카페,디저트
5
카스테라연구소
베이커리
6
무계획
카페,디저트
7
페이브먼트
카페,디저트
8
차차
카페,디저트
9
로우머
카페,디저트
10
온리디스베이커리
베이커리
1

In [7]:
data_list= data[0]['카페정보']

In [8]:
import pandas as pd

df = pd.DataFrame(data_list)
df

,name,cafe_type
0,베이코닉 브런치바,브런치
1,라비뛰드,베이커리
2,어글리베이커리,베이커리
3,더바이글,베이커리
4,소설원 망원,"카페,디저트"
...,...,...
299,카페리 마포구민체육센터점,카페
300,해비치스터디카페,스터디카페
301,Cafe Miei,"카페,디저트"
302,초록북카페,북카페


In [9]:
unique_name= df['name'].value_counts()

In [14]:
duplicate_names= unique_name[unique_name > 1].index
duplicate_names

Index(['베이코닉 브런치바', '키토빵앗간', '마떼아', '라비뛰드'], dtype='object', name='name')

In [11]:
del_name=[]
for name in duplicate_names: 
    del_name.append(df[df['name'] == name].index[0])

In [12]:
df.drop(del_name, inplace=True)

In [13]:
df['name'].value_counts()

name
어글리베이커리         1
스튜디오파우드         1
카페게이트 망원역점      1
라디오망원           1
rora            1
               ..
모구모구과자점 인 망원    1
범골커피            1
WHH 와하하         1
픽셀              1
안흥찐빵            1
Name: count, Length: 300, dtype: int64

In [15]:
df.to_csv('cafelist_망원동.csv',encoding='utf-8-sig')

In [18]:
import json
import time
from time import sleep

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

# --크롬창을 숨기고 실행-- driver에 options를 추가해주면된다
# options = webdriver.ChromeOptions()
# options.add_argument('headless')

url = 'https://map.naver.com/v5/search'
driver = webdriver.Chrome()  # 드라이버 경로
# driver = webdriver.Chrome('./chromedriver',chrome_options=options) # 크롬창 숨기기
driver.get(url)
key_word = '연남동 카페'  # 검색어

# css 찾을때 까지 10초대기
def time_wait(num, code):
    try:
        wait = WebDriverWait(driver, num).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, code)))
    except:
        print(code, '태그를 찾지 못하였습니다.')
        driver.quit()
    return wait

# frame 변경 메소드
def switch_frame(frame):
    driver.switch_to.default_content()  # frame 초기화
    driver.switch_to.frame(frame)  # frame 변경

# 페이지 다운
def page_down(num):
    body = driver.find_element(By.CSS_SELECTOR, 'body')
    body.click()
    for i in range(num):
        body.send_keys(Keys.PAGE_DOWN)

# css를 찾을때 까지 10초 대기
time_wait(10, 'div.input_box > input.input_search')

# (1) 검색창 찾기
search = driver.find_element(By.CSS_SELECTOR, 'div.input_box > input.input_search')
search.send_keys(key_word)  # 검색어 입력
search.send_keys(Keys.ENTER)  # 엔터버튼 누르기

sleep(1)

# (2) frame 변경
switch_frame('searchIframe')
page_down(40)
sleep(3)

# 카페 리스트
cafe_list = driver.find_elements(By.CSS_SELECTOR, 'li.UEzoS')
# 페이지 리스트
next_btn = driver.find_elements(By.CSS_SELECTOR, '.zRM9F> a')

# dictionary 생성
cafe_dict = {'카페정보': []}
# 시작시간
start = time.time()
print('[크롤링 시작...]')

# 크롤링 (페이지 리스트 만큼)
for btn in range(len(next_btn))[1:]:  # next_btn[0] = 이전 페이지 버튼 무시 -> [1]부터 시작
    cafe_list = driver.find_elements(By.CSS_SELECTOR, 'li.UEzoS')
    names = driver.find_elements(By.CSS_SELECTOR, '.TYaxT')  # (3) 장소명
    types = driver.find_elements(By.CSS_SELECTOR, '.KCMnt')  # (4) 장소 유형

    for data in range(len(cafe_list)):  # 카페 리스트 만큼
        print(data)

        sleep(1)
        try:
            # 지번, 도로명 초기화
            jibun_address = ''
            road_address = ''

            # (3) 카페명 가져오기
            cafe_name = names[data].text
            print(cafe_name)

            # (4) 유형
            cafe_type = types[data].text
            print(cafe_type)

            # dict에 데이터 집어넣기
            dict_temp = {
                'name': cafe_name,
                'cafe_type': cafe_type,
                # 'road_address': road_address,
                # 'jibun_address': jibun_address
            }

            cafe_dict['카페정보'].append(dict_temp)
            sleep(1)

        except Exception as e:
            print(e)
            print('ERROR!' * 3)

            dict_temp = {
                'name': cafe_name,
                'cafe_type': cafe_type

            }

            cafe_dict['카페정보'].append(dict_temp)
            sleep(1)

    # 다음 페이지 버튼 누를 수 없으면 종료
    if not next_btn[-1].is_enabled():
        break

    if names[-1]:  # 마지막 카페일 경우 다음버튼 클릭
        next_btn[-1].click()

        sleep(2)

    else:
        print('페이지 인식 못함')
        break

print('[데이터 수집 완료]\n소요 시간 :', time.time() - start)
driver.quit()  # 작업이 끝나면 창을 닫는다.

# 배열에 저장
data = [cafe_dict]


[크롤링 시작...]
0
사이드테이블
카페,디저트
1
잼잼
카페,디저트
2
브레디포스트 연남
카페,디저트
3
팽페르뒤
카페,디저트
4
코코로카라
베이커리
5
터틀힙
카페,디저트
6
유키모찌 연남점
아이스크림
7
조앤도슨
카페,디저트
8
오츠커피 연남
카페,디저트
9
작당모의
카페,디저트
0
씨스루
카페,디저트
1
오흐뒤구떼
베이커리
2
수크레크레페하우스
브런치카페
3
그림카페
카페,디저트
4
페블스
카페
5
멜팅오브
카페
6
턴다운서비스
카페,디저트
7
리틀빅토리
베이커리
8
SOP
카페,디저트
9
벌스가든
카페,디저트
0
코코시에나 티오마카세
차
1
스몰럭케이크
카페,디저트
2
위아소울메이트
카페,디저트
3
누아즈데테
카페,디저트
4
유니드마이요거트
카페,디저트
5
녹온
카페,디저트
6
시계
카페,디저트
7
피스피스 연남점
카페,디저트
8
차차커피코 연남점
카페,디저트
9
커피냅로스터스 연남동
카페
0
휘퍼스
카페,디저트
1
벨라쿠키 연남
카페,디저트
2
그림제작소 연남점
테마카페
3
코발트무드
카페,디저트
4
환대
카페,디저트
5
디그이지
카페,디저트
6
연남살롱
카페
7
딥스
카페,디저트
8
카페하이웨스트 연남점
카페,디저트
9
렌테
카페,디저트
0
지구샵 제로웨이스트홈
베이커리
1
어나더쉐어
카페,디저트
2
탕후루Hub
카페,디저트
3
데저트
카페,디저트
4
카페 어쿠스틱
카페,디저트
5
부빵
베이커리
6
도 도넛
도넛
7
메리고라운드 연남점
카페,디저트
8
카페몽슈슈 연남점
베이커리
9
포스트 케이크
케이크전문
0
유메이크쿠키
카페,디저트
1
컴포넌트 커피
카페,디저트
2
바람커피원두상점
카페,디저트
3
멜티스트
카페,디저트
4
홍대츄러스
카페,디저트
5
백종명화이트커피
카페,디저트
6
마르셀
카페,디저트
7
프롬라떼 연남
카페,디저트
8
연나방
카페,디저트
9
플루프커피
카페,디저트
[데이터 수집 완료]
소요 시간 : 140.27215456962585


In [19]:
data_list= data[0]['카페정보']
import pandas as pd

df = pd.DataFrame(data_list)
df
unique_name= df['name'].value_counts()
duplicate_names= unique_name[unique_name > 1].index
duplicate_names
del_name=[]
for name in duplicate_names: 
    del_name.append(df[df['name'] == name].index[0])

df.drop(del_name, inplace=True)
df['name'].value_counts()
df.to_csv('cafelist_연남동.csv',encoding='utf-8-sig')

In [2]:
import json
import time
from time import sleep

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

# --크롬창을 숨기고 실행-- driver에 options를 추가해주면된다
# options = webdriver.ChromeOptions()
# options.add_argument('headless')

url = 'https://map.naver.com/v5/search'
driver = webdriver.Chrome()  # 드라이버 경로
# driver = webdriver.Chrome('./chromedriver',chrome_options=options) # 크롬창 숨기기
driver.get(url)
key_word = '한남동 카페'  # 검색어

# css 찾을때 까지 10초대기
def time_wait(num, code):
    try:
        wait = WebDriverWait(driver, num).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, code)))
    except:
        print(code, '태그를 찾지 못하였습니다.')
        driver.quit()
    return wait

# frame 변경 메소드
def switch_frame(frame):
    driver.switch_to.default_content()  # frame 초기화
    driver.switch_to.frame(frame)  # frame 변경

# 페이지 다운
def page_down(num):
    body = driver.find_element(By.CSS_SELECTOR, 'body')
    body.click()
    for i in range(num):
        body.send_keys(Keys.PAGE_DOWN)

# css를 찾을때 까지 10초 대기
time_wait(10, 'div.input_box > input.input_search')

# (1) 검색창 찾기
search = driver.find_element(By.CSS_SELECTOR, 'div.input_box > input.input_search')
search.send_keys(key_word)  # 검색어 입력
search.send_keys(Keys.ENTER)  # 엔터버튼 누르기

sleep(1)

# (2) frame 변경
switch_frame('searchIframe')
page_down(40)
sleep(3)

# 카페 리스트
cafe_list = driver.find_elements(By.CSS_SELECTOR, 'li.UEzoS')
# 페이지 리스트
next_btn = driver.find_elements(By.CSS_SELECTOR, '.zRM9F> a')

# dictionary 생성
cafe_dict = {'카페정보': []}
# 시작시간
start = time.time()
print('[크롤링 시작...]')

# 크롤링 (페이지 리스트 만큼)
for btn in range(len(next_btn))[1:]:  # next_btn[0] = 이전 페이지 버튼 무시 -> [1]부터 시작
    cafe_list = driver.find_elements(By.CSS_SELECTOR, 'li.UEzoS')
    names = driver.find_elements(By.CSS_SELECTOR, '.TYaxT')  # (3) 장소명
    types = driver.find_elements(By.CSS_SELECTOR, '.KCMnt')  # (4) 장소 유형

    for data in range(len(cafe_list)):  # 카페 리스트 만큼
        print(data)

        sleep(1)
        try:
            # 지번, 도로명 초기화
            jibun_address = ''
            road_address = ''

            # (3) 카페명 가져오기
            cafe_name = names[data].text
            print(cafe_name)

            # (4) 유형
            cafe_type = types[data].text
            print(cafe_type)

            # dict에 데이터 집어넣기
            dict_temp = {
                'name': cafe_name,
                'cafe_type': cafe_type,
                # 'road_address': road_address,
                # 'jibun_address': jibun_address
            }

            cafe_dict['카페정보'].append(dict_temp)
            sleep(1)

        except Exception as e:
            print(e)
            print('ERROR!' * 3)

            dict_temp = {
                'name': cafe_name,
                'cafe_type': cafe_type

            }

            cafe_dict['카페정보'].append(dict_temp)
            sleep(1)

    # 다음 페이지 버튼 누를 수 없으면 종료
    if not next_btn[-1].is_enabled():
        break

    if names[-1]:  # 마지막 카페일 경우 다음버튼 클릭
        next_btn[-1].click()

        sleep(2)

    else:
        print('페이지 인식 못함')
        break

print('[데이터 수집 완료]\n소요 시간 :', time.time() - start)
driver.quit()  # 작업이 끝나면 창을 닫는다.

# 배열에 저장
data = [cafe_dict]

[크롤링 시작...]
0
AVEK CHERI
카페,디저트
1
클럽하우스서울
카페,디저트
2
올드페리도넛 한남점
도넛
3
골든피스
카페,디저트
4
마일스톤 커피 한남점
카페
5
올댓재즈
라이브카페
6
데이로우 DAY ROW
카페,디저트
7
맥심플랜트
카페
8
패션5
베이커리
9
네로우 패스
카페,디저트
10
콘하스 한남점
카페,디저트
11
엔다이브
카페,디저트
12
그랜드하얏트서울 델리
베이커리
13
한남작업실
카페,디저트
14
아티장베이커스 한남
베이커리
15
재인
카페,디저트
16
한남베르그
카페,디저트
17
르솔레이
카페,디저트
18
원인어밀리언
카페,디저트
19
mtl 한남
카페,디저트
20
보닐라츄러스 한남점
카페,디저트
21
울프소셜클럽
카페,디저트
22
한남 상석
요리주점
23
케르반베이커리&카페
베이커리
24
오지힐
카페,디저트
25
AVEK CHERI
카페,디저트
26
타르틴 베이커리 한남점
베이커리
27
그랜드하얏트서울 갤러리
카페,디저트
28
블루보틀 한남 카페
블루보틀
29
프렌떼 한남
베이커리
30
웻커피 한남
카페,디저트
31
히트커피로스터스 한남
카페,디저트
32
104
카페,디저트
33
베베베 한남점
베이글
34
오아시스 한남점
브런치
35
베이커리 오월의종
베이커리
36
레종데트르 한남
카페,디저트
37
산수화티하우스
차
38
릴리언
베이커리
39
수르기
카페,디저트
40
아키비스트 한남
카페,디저트
41
콘디토리 오븐
카페,디저트
42
허니비 서울
카페,디저트
43
베베르카
브런치
44
크레이트커피
카페
45
아우프글렛 한남
카페,디저트
46
한남라메종
카페,디저트
47
팥알로
베이커리
48
카라멜리에오
베이커리
49
로우커피
카페,디저트
50
아이스크림 소사이어티
아이스크림
51
북파크라운지
북카페
52
아우어베이커리 한남나인원점
베이커리
53
배스킨라빈스 하이브한남점
아이스크림
0
테라로사 한남점
카페
1
Summer Lane
브런치
2
폴바셋 한남 커피스테이션점
카페
3
뚜띠한남
카페,디저트
4
맥코이 한남
카페,디저트
5


In [3]:
data_list= data[0]['카페정보']
import pandas as pd

df = pd.DataFrame(data_list)
df
unique_name= df['name'].value_counts()
duplicate_names= unique_name[unique_name > 1].index
duplicate_names
del_name=[]
for name in duplicate_names: 
    del_name.append(df[df['name'] == name].index[0])

df.drop(del_name, inplace=True)
df['name'].value_counts()
df.to_csv('cafelist_한남동.csv',encoding='utf-8-sig')

In [22]:
import json
import time
from time import sleep

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

# --크롬창을 숨기고 실행-- driver에 options를 추가해주면된다
# options = webdriver.ChromeOptions()
# options.add_argument('headless')

url = 'https://map.naver.com/v5/search'
driver = webdriver.Chrome()  # 드라이버 경로
# driver = webdriver.Chrome('./chromedriver',chrome_options=options) # 크롬창 숨기기
driver.get(url)
key_word = '성수동 카페'  # 검색어

# css 찾을때 까지 10초대기
def time_wait(num, code):
    try:
        wait = WebDriverWait(driver, num).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, code)))
    except:
        print(code, '태그를 찾지 못하였습니다.')
        driver.quit()
    return wait

# frame 변경 메소드
def switch_frame(frame):
    driver.switch_to.default_content()  # frame 초기화
    driver.switch_to.frame(frame)  # frame 변경

# 페이지 다운
def page_down(num):
    body = driver.find_element(By.CSS_SELECTOR, 'body')
    body.click()
    for i in range(num):
        body.send_keys(Keys.PAGE_DOWN)

# css를 찾을때 까지 10초 대기
time_wait(10, 'div.input_box > input.input_search')

# (1) 검색창 찾기
search = driver.find_element(By.CSS_SELECTOR, 'div.input_box > input.input_search')
search.send_keys(key_word)  # 검색어 입력
search.send_keys(Keys.ENTER)  # 엔터버튼 누르기

sleep(1)

# (2) frame 변경
switch_frame('searchIframe')
page_down(40)
sleep(3)

# 카페 리스트
cafe_list = driver.find_elements(By.CSS_SELECTOR, 'li.UEzoS')
# 페이지 리스트
next_btn = driver.find_elements(By.CSS_SELECTOR, '.zRM9F> a')

# dictionary 생성
cafe_dict = {'카페정보': []}
# 시작시간
start = time.time()
print('[크롤링 시작...]')

# 크롤링 (페이지 리스트 만큼)
for btn in range(len(next_btn))[1:]:  # next_btn[0] = 이전 페이지 버튼 무시 -> [1]부터 시작
    cafe_list = driver.find_elements(By.CSS_SELECTOR, 'li.UEzoS')
    names = driver.find_elements(By.CSS_SELECTOR, '.TYaxT')  # (3) 장소명
    types = driver.find_elements(By.CSS_SELECTOR, '.KCMnt')  # (4) 장소 유형

    for data in range(len(cafe_list)):  # 카페 리스트 만큼
        print(data)

        sleep(1)
        try:
            # 지번, 도로명 초기화
            jibun_address = ''
            road_address = ''

            # (3) 카페명 가져오기
            cafe_name = names[data].text
            print(cafe_name)

            # (4) 유형
            cafe_type = types[data].text
            print(cafe_type)

            # dict에 데이터 집어넣기
            dict_temp = {
                'name': cafe_name,
                'cafe_type': cafe_type,
                # 'road_address': road_address,
                # 'jibun_address': jibun_address
            }

            cafe_dict['카페정보'].append(dict_temp)
            sleep(1)

        except Exception as e:
            print(e)
            print('ERROR!' * 3)

            dict_temp = {
                'name': cafe_name,
                'cafe_type': cafe_type

            }

            cafe_dict['카페정보'].append(dict_temp)
            sleep(1)

    # 다음 페이지 버튼 누를 수 없으면 종료
    if not next_btn[-1].is_enabled():
        break

    if names[-1]:  # 마지막 카페일 경우 다음버튼 클릭
        next_btn[-1].click()

        sleep(2)

    else:
        print('페이지 인식 못함')
        break

print('[데이터 수집 완료]\n소요 시간 :', time.time() - start)
driver.quit()  # 작업이 끝나면 창을 닫는다.

# 배열에 저장
data = [cafe_dict]

[크롤링 시작...]
0
와아
와인
1
침묵을깨다
와인
2
차일디쉬
카페,디저트
3
자연도소금빵in 성수
베이커리
4
한정선
카페,디저트
5
치즈인더스트리 성수
카페,디저트
6
츄로101 성수점
카페
7
모리노키세츠
카페,디저트
8
하이라인
카페,디저트
9
오우드 성수1호점
카페,디저트
10
베통 성수
카페,디저트
11
파머스카페
카페,디저트
12
브레디포스트 성수
카페,디저트
13
웜브라운
카페,디저트
14
니커버커베이글 성수지점
베이커리
15
본지르르 성수
카페,디저트
16
마를리
케이크전문
17
아쿠아산타 성수카페
카페,디저트
18
서울앵무새
카페,디저트
19
크림라벨 서울숲 본점
케이크전문
20
쎈느Scene
카페,디저트
21
카페 할아버지공장
카페,디저트
22
기미사 성수
카페
23
디데이크
케이크전문
24
스트로베리32 성수점
베이커리
25
퍼먼트
베이커리
26
성수 베이킹 스튜디오
베이커리
27
어니언 성수
카페,디저트
28
밀도 성수점
밀도
29
LOOOP 루프 베이커리 카페
카페
30
피넛버터바나나
카페,디저트
31
비아트 성수
카페,디저트
32
체다앤올리
카페,디저트
33
실버리
카페,디저트
34
묘사서울
카페
35
라프레플루트
카페,디저트
36
누데이크 성수
카페,디저트
37
LOWIDE 서울숲
카페,디저트
38
맥파이앤타이거 성수티룸
카페,디저트
39
브라우터
카페
40
코끼리베이글 성수
베이커리
41
러프러프
카페,디저트
42
레인리포트 크루아상
카페,디저트
43
뵈르뵈르
아이스크림
44
구욱희씨 서울숲 본점
카페,디저트
45
블루보틀 성수 카페
블루보틀
46
커피냅로스터스 성수
카페,디저트
47
봉땅 서울숲점
베이커리
48
로우키
카페
49
5to7
카페,디저트
50
플디 성수점
카페,디저트
51
꼬모윤 서울숲점
카페,디저트
52
텅 성수 스페이스
카페,디저트
53
뉴앳던 성수
카페,디저트
0
에이투비 Cafe&Bar
카페,디저트
1
발렁스
카페,디저트
2
치카성수
케이크전문
3
디플랫 성수
카페,디저트
4
목로정원
카페,디저트
5
테

In [24]:
data_list= data[0]['카페정보']
import pandas as pd

df = pd.DataFrame(data_list)
df
unique_name= df['name'].value_counts()
duplicate_names= unique_name[unique_name > 1].index
duplicate_names
del_name=[]
for name in duplicate_names: 
    del_name.append(df[df['name'] == name].index[0])

df.drop(del_name, inplace=True)
df['name'].value_counts()
df.to_csv('cafelist_성수동.csv',encoding='utf-8-sig')